In [39]:
import pandas as pd
import plotly.express as px
from datetime import timedelta, datetime
import math
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

In [40]:
base_data = pd.read_csv("../datasets/agoda_cancellation_train.csv")
base_data.booking_datetime= pd.to_datetime(base_data['booking_datetime'])
base_data.checkout_date= pd.to_datetime(base_data['checkout_date'])
base_data.checkin_date= pd.to_datetime(base_data['checkin_date'])
base_data.cancellation_datetime= pd.to_datetime(base_data['cancellation_datetime'])
base_data.describe().T

,count,mean,std,min,25%,50%,75%,max
h_booking_id,58659.0,-1.906361e+16,5.330681e+18,-9.223194e+18,-4.632019e+18,-2.461317e+16,4.574774e+18,9.223338e+18
hotel_id,58659.0,1.345665e+06,1.363885e+06,1.000000e+00,2.560885e+05,8.050080e+05,2.284811e+06,5.823993e+06
hotel_star_rating,58659.0,3.221824e+00,1.173631e+00,-1.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00
h_customer_id,58659.0,4.506973e+18,2.805723e+18,-9.096408e+18,2.223609e+18,4.524707e+18,6.872115e+18,9.223335e+18
guest_is_not_the_customer,58659.0,2.148690e-01,4.107349e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
no_of_adults,58659.0,2.345079e+00,1.326385e+00,1.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,3.800000e+01
no_of_children,58659.0,1.484342e-01,5.307000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+01
no_of_extra_bed,58659.0,1.147309e-02,1.158512e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00
no_of_room,58659.0,1.139655e+00,5.164938e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.000000e+00
original_selling_amount,58659.0,2.190191e+02,4.519661e+02,2.100000e+00,5.021500e+01,1.076100e+02,2.422650e+02,4.956616e+04


In [41]:
test_data = pd.read_csv("../datasets/test_set_week_1.csv")
test_data.booking_datetime= pd.to_datetime(test_data['booking_datetime'])
test_data.checkout_date= pd.to_datetime(test_data['checkout_date'])
test_data.checkin_date= pd.to_datetime(test_data['checkin_date'])
test_data.describe().T

,count,mean,std,min,25%,50%,75%,max
h_booking_id,700.0,-8.402476e+18,4.729080e+17,-9.221607e+18,-8.832934e+18,-8.367710e+18,-8.009815e+18,-7.608677e+18
hotel_id,700.0,1.531205e+06,1.578884e+06,3.780000e+02,2.462430e+05,9.012885e+05,2.645298e+06,5.875678e+06
hotel_star_rating,700.0,3.277857e+00,1.068610e+00,0.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00
h_customer_id,700.0,4.527987e+18,2.843795e+18,-7.586689e+18,2.213578e+18,4.555195e+18,6.937259e+18,9.220768e+18
guest_is_not_the_customer,700.0,1.757143e-01,3.808490e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
no_of_adults,700.0,2.552857e+00,1.406629e+00,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.600000e+01
no_of_children,700.0,2.642857e-01,7.157608e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+00
no_of_extra_bed,700.0,8.571429e-03,9.225029e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
no_of_room,700.0,1.230000e+00,6.462946e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,8.000000e+00
original_selling_amount,700.0,3.183699e+02,4.406012e+02,9.000000e+00,8.887500e+01,1.891900e+02,3.661950e+02,5.171100e+03


In [21]:
sd = datetime(2018, 12, 13)
relevant_test_data = test_data.loc[test_data.booking_datetime <= sd]
relevant_test_data.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,h_customer_id,customer_nationality,guest_is_not_the_customer,guest_nationality_country_name,no_of_adults,no_of_children,no_of_extra_bed,no_of_room,origin_country_code,language,original_selling_amount,original_payment_method,original_payment_type,original_payment_currency,is_user_logged_in,cancellation_policy_code,is_first_booking,request_nonesmoke,request_latecheckin,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
0,-9221607263697497458,2018-11-26 10:57:00,2018-12-20,2018-12-21,3070197,IN,2017-11-02 11:54:00,3.0,Hotel,Pay Now,7281026750058370014,Australia,0,Australia,4,0,0,2,AU,English / Australia,89.32,MasterCard,Credit Card,AUD,False,2D100P,True,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2620,NaN,NaN,1853
1,-9218553471626512105,2018-11-30 11:45:00,2019-02-09,2019-02-11,706182,TH,2014-08-22 16:10:00,3.0,Resort,Pay Later,6223510701031070015,Thailand,0,Thailand,2,0,0,1,TH,Thai,135.36,Visa,Credit Card,THB,True,7D1N_100P,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5484,NaN,NaN,1241
2,-9217919187333201744,2018-11-25 21:44:00,2019-04-22,2019-04-23,400322,JP,2012-10-02 13:54:00,3.0,Hotel,Pay Later,4004179407299760023,Hong Kong,0,Hong Kong,4,0,0,2,HK,T. Chinese / Taiwan,215.04,MasterCard,Credit Card,HKD,True,3D100P_100P,False,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2884,193.0,344.0,2264
3,-9215699014988905507,2018-11-20 20:20:00,2018-12-16,2018-12-20,3135771,PH,2017-11-06 17:05:00,5.0,Resort,Pay Now,9105315817276280047,South Korea,0,South Korea,2,1,0,1,KR,Korean,930.67,American Express,Credit Card,KRW,False,27D100P,True,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2532,415.0,291.0,1405
4,-9212818888388326328,2018-11-28 19:11:00,2018-12-24,2018-12-26,5002155,TW,2018-06-01 18:19:00,5.0,Hotel,Pay Later,3892666891828110043,Japan,0,Japan,2,0,0,1,TW,Japanese,233.10,Visa,Credit Card,TWD,False,3D50P_100P,False,1.0,0.0,0.0,0.0,1.0,0.0,0.0,4903,NaN,NaN,135


In [22]:
base_data.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,h_customer_id,customer_nationality,guest_is_not_the_customer,guest_nationality_country_name,no_of_adults,no_of_children,no_of_extra_bed,no_of_room,origin_country_code,language,original_selling_amount,original_payment_method,original_payment_type,original_payment_currency,is_user_logged_in,cancellation_policy_code,is_first_booking,request_nonesmoke,request_latecheckin,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
0,-9223194055642672935,2018-06-28 21:15:00,2018-07-09,2018-07-13,6452,HK,2009-06-28 02:02:00,4.0,Hotel,Pay Now,6003835896819550002,China,0,China,2,0,0,1,CN,S.Chinese / Mainland,267.55,UNKNOWN,Invoice,CNY,False,365D100P_100P,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1192,NaN,NaN,142
1,-9222713784330706132,2018-08-10 22:31:00,2018-08-16,2018-08-17,47729,CN,2011-06-07 11:52:00,4.0,Hotel,Pay Later,6337727081989900092,South Africa,0,South Africa,2,0,0,1,ZA,English,114.15,Visa,Credit Card,ZAR,False,1D1N_1N,False,1.0,0.0,0.0,0.0,1.0,0.0,0.0,NaT,5617,902.0,600.0,1273
2,-9222411208325704942,2018-09-14 07:55:00,2018-09-14,2018-09-15,780431,KR,2014-11-20 15:43:00,4.0,Hotel,Pay Now,4569478218455470035,South Korea,0,South Korea,2,0,0,1,KR,Korean,115.79,MasterCard,Credit Card,KRW,False,2D1N_1N,True,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT,3819,NaN,NaN,2799
3,-9222220845872895471,2018-06-25 07:33:00,2018-07-02,2018-07-03,291365,JP,2011-11-21 12:27:00,3.0,Hotel,Pay Later,5223266395530630065,Singapore,0,Singapore,2,0,0,1,SG,English / United Kingdom,68.74,American Express,Credit Card,SGD,True,1D1N_100P,False,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaT,2379,466.0,236.0,1448
4,-9221127186162682116,2018-07-23 10:06:00,2018-08-09,2018-08-10,479046,TH,2013-06-06 10:04:00,3.0,Resort,Pay Later,2879166343687280017,Thailand,0,Thailand,8,0,0,2,TH,Thai,127.76,MasterCard,Credit Card,THB,False,14D100P_100P,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2018-07-23,1506,NaN,NaN,616


In [23]:
base_data['cancellation_datetime']

0              NaT
1              NaT
2              NaT
3              NaT
4       2018-07-23
           ...    
58654   2018-07-22
58655          NaT
58656          NaT
58657          NaT
58658          NaT
Name: cancellation_datetime, Length: 58659, dtype: datetime64[ns]

In [24]:
def get_canceled_days_before_checkout(canceled_date):
    if pd.isnull(canceled_date):
        return 0
    return 1
    

In [25]:
base_data['is_cancled'] = base_data.cancellation_datetime.apply(lambda t: get_canceled_days_before_checkout(t))

In [26]:
base_data['is_cancled']

0        0
1        0
2        0
3        0
4        1
        ..
58654    1
58655    0
58656    0
58657    0
58658    0
Name: is_cancled, Length: 58659, dtype: int64

In [27]:
sd = datetime(2018, 12, 13)
relevant_test_data = test_data.loc[test_data.booking_datetime <= sd]
relevant_test_data.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,h_customer_id,customer_nationality,guest_is_not_the_customer,guest_nationality_country_name,no_of_adults,no_of_children,no_of_extra_bed,no_of_room,origin_country_code,language,original_selling_amount,original_payment_method,original_payment_type,original_payment_currency,is_user_logged_in,cancellation_policy_code,is_first_booking,request_nonesmoke,request_latecheckin,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
0,-9221607263697497458,2018-11-26 10:57:00,2018-12-20,2018-12-21,3070197,IN,2017-11-02 11:54:00,3.0,Hotel,Pay Now,7281026750058370014,Australia,0,Australia,4,0,0,2,AU,English / Australia,89.32,MasterCard,Credit Card,AUD,False,2D100P,True,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2620,NaN,NaN,1853
1,-9218553471626512105,2018-11-30 11:45:00,2019-02-09,2019-02-11,706182,TH,2014-08-22 16:10:00,3.0,Resort,Pay Later,6223510701031070015,Thailand,0,Thailand,2,0,0,1,TH,Thai,135.36,Visa,Credit Card,THB,True,7D1N_100P,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5484,NaN,NaN,1241
2,-9217919187333201744,2018-11-25 21:44:00,2019-04-22,2019-04-23,400322,JP,2012-10-02 13:54:00,3.0,Hotel,Pay Later,4004179407299760023,Hong Kong,0,Hong Kong,4,0,0,2,HK,T. Chinese / Taiwan,215.04,MasterCard,Credit Card,HKD,True,3D100P_100P,False,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2884,193.0,344.0,2264
3,-9215699014988905507,2018-11-20 20:20:00,2018-12-16,2018-12-20,3135771,PH,2017-11-06 17:05:00,5.0,Resort,Pay Now,9105315817276280047,South Korea,0,South Korea,2,1,0,1,KR,Korean,930.67,American Express,Credit Card,KRW,False,27D100P,True,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2532,415.0,291.0,1405
4,-9212818888388326328,2018-11-28 19:11:00,2018-12-24,2018-12-26,5002155,TW,2018-06-01 18:19:00,5.0,Hotel,Pay Later,3892666891828110043,Japan,0,Japan,2,0,0,1,TW,Japanese,233.10,Visa,Credit Card,TWD,False,3D50P_100P,False,1.0,0.0,0.0,0.0,1.0,0.0,0.0,4903,NaN,NaN,135


In [28]:
base_data.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,h_customer_id,customer_nationality,guest_is_not_the_customer,guest_nationality_country_name,no_of_adults,no_of_children,no_of_extra_bed,no_of_room,origin_country_code,language,original_selling_amount,original_payment_method,original_payment_type,original_payment_currency,is_user_logged_in,cancellation_policy_code,is_first_booking,request_nonesmoke,request_latecheckin,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code,is_cancled
0,-9223194055642672935,2018-06-28 21:15:00,2018-07-09,2018-07-13,6452,HK,2009-06-28 02:02:00,4.0,Hotel,Pay Now,6003835896819550002,China,0,China,2,0,0,1,CN,S.Chinese / Mainland,267.55,UNKNOWN,Invoice,CNY,False,365D100P_100P,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1192,NaN,NaN,142,0
1,-9222713784330706132,2018-08-10 22:31:00,2018-08-16,2018-08-17,47729,CN,2011-06-07 11:52:00,4.0,Hotel,Pay Later,6337727081989900092,South Africa,0,South Africa,2,0,0,1,ZA,English,114.15,Visa,Credit Card,ZAR,False,1D1N_1N,False,1.0,0.0,0.0,0.0,1.0,0.0,0.0,NaT,5617,902.0,600.0,1273,0
2,-9222411208325704942,2018-09-14 07:55:00,2018-09-14,2018-09-15,780431,KR,2014-11-20 15:43:00,4.0,Hotel,Pay Now,4569478218455470035,South Korea,0,South Korea,2,0,0,1,KR,Korean,115.79,MasterCard,Credit Card,KRW,False,2D1N_1N,True,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT,3819,NaN,NaN,2799,0
3,-9222220845872895471,2018-06-25 07:33:00,2018-07-02,2018-07-03,291365,JP,2011-11-21 12:27:00,3.0,Hotel,Pay Later,5223266395530630065,Singapore,0,Singapore,2,0,0,1,SG,English / United Kingdom,68.74,American Express,Credit Card,SGD,True,1D1N_100P,False,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaT,2379,466.0,236.0,1448,0
4,-9221127186162682116,2018-07-23 10:06:00,2018-08-09,2018-08-10,479046,TH,2013-06-06 10:04:00,3.0,Resort,Pay Later,2879166343687280017,Thailand,0,Thailand,8,0,0,2,TH,Thai,127.76,MasterCard,Credit Card,THB,False,14D100P_100P,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2018-07-23,1506,NaN,NaN,616,1


In [29]:
base_data['cancellation_datetime']

0              NaT
1              NaT
2              NaT
3              NaT
4       2018-07-23
           ...    
58654   2018-07-22
58655          NaT
58656          NaT
58657          NaT
58658          NaT
Name: cancellation_datetime, Length: 58659, dtype: datetime64[ns]

In [30]:
def get_canceled_days_before_checkout(canceled_date):
    if pd.isnull(canceled_date):
        return 0
    return 1
    

In [31]:
base_data['is_cancled'] = base_data.cancellation_datetime.apply(lambda t: get_canceled_days_before_checkout(t))

In [32]:
base_data['is_cancled']

0        0
1        0
2        0
3        0
4        1
        ..
58654    1
58655    0
58656    0
58657    0
58658    0
Name: is_cancled, Length: 58659, dtype: int64

In [51]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)
new_data = pd.read_csv("../datasets/house_prices.csv")
new_data.corr()


,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,1.000000,-0.016648,0.001797,0.005531,-0.011981,-0.132023,0.018827,-0.002707,0.011619,-0.023372,0.008574,-0.010778,-0.005048,0.023883,-0.016888,0.010141,0.009245,-0.007207,-0.002789,-0.138706
price,-0.016648,1.000000,0.308254,0.525184,0.701926,0.089864,0.257031,0.266288,0.396768,0.036516,0.667088,0.605565,0.323530,0.053675,0.126437,0.005710,0.130016,-0.006231,0.585336,0.082740
bedrooms,0.001797,0.308254,1.000000,0.516323,0.576955,0.031799,0.176211,-0.006552,0.079548,0.029303,0.357621,0.477782,0.303082,0.150483,0.018864,0.012364,0.019152,-0.002677,0.391948,0.029315
bathrooms,0.005531,0.525184,0.516323,1.000000,0.754807,0.087791,0.501022,0.063739,0.187728,-0.124237,0.665055,0.685433,0.283769,0.468033,0.050748,0.002318,0.026855,0.018839,0.568801,0.087187
sqft_living,-0.011981,0.701926,0.576955,0.754807,1.000000,0.172843,0.354351,0.103804,0.284584,-0.058190,0.762562,0.876630,0.435005,0.295744,0.055366,-0.000590,0.034858,0.024937,0.756491,0.183309
sqft_lot,-0.132023,0.089864,0.031799,0.087791,0.172843,1.000000,-0.005115,0.021607,0.074718,-0.008858,0.113604,0.183501,0.015304,0.049303,0.007649,-0.007885,-0.031459,0.035889,0.144586,0.718556
floors,0.018827,0.257031,0.176211,0.501022,0.354351,-0.005115,1.000000,0.023708,0.029464,-0.262903,0.458512,0.524039,-0.245595,0.452909,0.006355,0.014033,0.036848,0.002393,0.280212,-0.011195
waterfront,-0.002707,0.266288,-0.006552,0.063739,0.103804,0.021607,0.023708,1.000000,0.401858,0.016666,0.082713,0.072065,0.080592,-0.023557,0.092886,0.003012,-0.005077,-0.007585,0.086438,0.030708
view,0.011619,0.396768,0.079548,0.187728,0.284584,0.074718,0.029464,0.401858,1.000000,0.046043,0.251141,0.167629,0.276958,-0.047768,0.103923,0.008855,0.004330,-0.015147,0.280356,0.072591
condition,-0.023372,0.036516,0.029303,-0.124237,-0.058190,-0.008858,-0.262903,0.016666,0.046043,1.000000,-0.142891,-0.157609,0.174168,-0.313813,-0.060522,0.035744,0.026847,-0.052895,-0.092101,-0.003305


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7.129301e+09,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0.0,3.0,7.0,1180.0,0.0,1955.0,0.0,98178.0,47.5112,-122.257,1340.0,5650.0
1,6.414100e+09,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0.0,3.0,7.0,2170.0,400.0,1951.0,1991.0,98125.0,47.7210,-122.319,1690.0,7639.0
2,5.631500e+09,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0.0,3.0,6.0,770.0,0.0,1933.0,0.0,98028.0,47.7379,-122.233,2720.0,8062.0
3,2.487201e+09,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0.0,5.0,7.0,1050.0,910.0,1965.0,0.0,98136.0,47.5208,-122.393,1360.0,5000.0
4,1.954401e+09,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0.0,3.0,8.0,1680.0,0.0,1987.0,0.0,98074.0,47.6168,-122.045,1800.0,7503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21611,2.630000e+08,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0.0,3.0,8.0,1530.0,0.0,2009.0,0.0,98103.0,47.6993,-122.346,1530.0,1509.0
21612,6.600060e+09,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0.0,3.0,8.0,2310.0,0.0,2014.0,0.0,98146.0,47.5107,-122.362,1830.0,7200.0
21613,1.523300e+09,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0.0,3.0,7.0,1020.0,0.0,2009.0,0.0,98144.0,47.5944,-122.299,1020.0,2007.0
21614,2.913101e+08,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0.0,3.0,8.0,1600.0,0.0,2004.0,0.0,98027.0,47.5345,-122.069,1410.0,1287.0
